In [89]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import os
import cv2
import torch
import torch.nn as nn
import torchvision
import numpy as np
import pandas as pd
base_dir = os.getcwd()

In [90]:
munich_imgs = os.listdir("scraped_images_grayscaled_big")
furnitures = [r for r in munich_imgs if "linz" and "cp13150" in r ]
furnitures

['1949_33905_id=cp131507_linz.jpg',
 '0711_4263_id=cp131504_linz.jpg',
 '0271_1458_id=cp131506_linz.jpg',
 '0606_3010_id=cp131502_linz.jpg',
 '0791_4970_id=cp131503_linz.jpg',
 '1950_33912_id=cp131509_linz.jpg',
 '1141_8563_id=cp131500_linz.jpg',
 '0270_1455_id=cp131505_linz.jpg',
 '1153_8691_id=cp131501_linz.jpg',
 '1950_33908_id=cp131508_linz.jpg']

In [83]:
# #vgg models (16 and 19 similar, but 16 is better)
# model = torchvision.models.vgg16(pretrained=True)
# # model = torchvision.models.vgg19(pretrained=True)

# model.features[0] = nn.Conv2d(1,64,kernel_size=(3,3), stride=(1,1),padding=(1,1))
# model = nn.Sequential(*[*list(model.children())[:-1][0][:-10]])

# def preprocess_image(image_path):
#     """
#     This function takes a path to a single image, it then resizes it to size 50x50 \
#     and normalizes it to the range [0,1]. Lastly, it adds an extra dimension to the image \
#     which represents the batch size. These steps are needed, because we want to pass the image \
#     to a CNN. 
#     """

#     # Read the image in grayscale mode
#     img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
#     if img is None:
#         raise ValueError(f"Image at path {image_path} could not be loaded.")
    
#     # Apply CLAHE (histogram equalization)
#     clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8,8))
#     img = clahe.apply(img)

#     # Apply thresholding
#     _, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
#     contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     contours = sorted(contours, key=cv2.contourArea, reverse=True)
#     mask = np.ones(img.shape, np.uint8) * 255
#     cv2.drawContours(mask, contours, 0, 0, -1)
#     img = cv2.add(thresh, mask)

#     # Apply erosion
#     kernel = np.ones((5,5), dtype=np.uint8)
#     img = cv2.erode(img, kernel, 10)
#     img = np.abs(np.max(img) - img)

#     # Apply gamma correction
#     img = np.power(img / 255.0, 2.0) * 255.0
#     img = np.clip(img, 0, 255).astype(np.uint8)

#     # Resize image to 50x50
#     img = cv2.resize(img, (50, 50), interpolation=cv2.INTER_AREA)
    
#     # Convert image to tensor
#     preprocess = torchvision.transforms.Compose([
#         torchvision.transforms.ToTensor()
#     ])
#     img = preprocess(img).unsqueeze(0)
#     return img
# def extract_features(image_path):
#     """
#     This function takes a path to a single image, it then preprocesses the image with the \
#     function preprocess_image. Afterwards it passes the image to the pretrained CNN to extract \
#     a feature descriptor. 
#     """
    
#     img = preprocess_image(image_path)
#     with torch.no_grad():
#         features = model(img)
#     return features.squeeze(0).numpy()

# def normalize_features(features):
#     """
#     This function takes the feature descriptor and normalizes it. This is needed as we want \
#     to compute the dot-product similarity between feature descriptors of different images. \
#     And for similarity it is convenient to have all pixels on the same scale without too \
#     much magnitude differences and this also ensures stability. 
#     """
    
#     return features / np.linalg.norm(features)
    

/home/jonathan/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jonathan/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
# # Resnet models (gives high scores regardless)
# # model = torchvision.models.resnet50(pretrained=True)
# # model = torchvision.models.resnet101(pretrained=True) 
# # model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
# # model = nn.Sequential(*list(model.children())[:-1])

# def preprocess_image(image_path):
#     """
#     This function takes a path to a single image, it then resizes it to size 50x50 \
#     and normalizes it to the range [0,1]. Lastly, it adds an extra dimension to the image \
#     which represents the batch size. These steps are needed, because we want to pass the image \
#     to a CNN. 
#     """
    
#     img = cv2.imread(image_path, -1)
#     clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8,8))
#     img = clahe.apply(img)
#     _, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
#     contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     contours = sorted(contours, key=cv2.contourArea, reverse=True)
#     mask = np.ones(img.shape, np.uint8)
#     mask.fill(255)
#     cv2.drawContours(mask, contours, 0, 0, -1)
#     img = cv2.add(thresh, mask)
#     kernel = np.ones((5,5), dtype=np.uint8)
#     img = cv2.erode(img, kernel, 10)
#     img = np.abs(np.max(img) - img)
    
#     img = cv2.resize(img, (50, 50), interpolation=cv2.INTER_AREA)
#     preprocess = torchvision.transforms.Compose([
#         torchvision.transforms.ToTensor()
#     ])
#     img = preprocess(img).unsqueeze(0)
#     return img

# def extract_features(image_path):
#     """
#     This function takes a path to a single image, it then preprocesses the image with the \
#     function preprocess_image. Afterwards it passes the image to the pretrained CNN to extract \
#     a feature descriptor. 
#     """
    
#     img = preprocess_image(image_path)
#     with torch.no_grad():
#         features = model(img)
#     return features.squeeze(0).numpy()

# def normalize_features(features):
#     """
#     This function takes the feature descriptor and normalizes it. This is needed as we want \
#     to compute the dot-product similarity between feature descriptors of different images. \
#     And for similarity it is convenient to have all pixels on the same scale without too \
#     much magnitude differences and this also ensures stability. 
#     """
    
#     return features / np.linalg.norm(features)
    

In [91]:
# InceptionV3 model (potential)
model = torchvision.models.inception_v3(pretrained=True, aux_logits=True)

def preprocess_image(image_path):
    """
    Preprocesses a single image for InceptionV3 input.
    """
    # img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read the image as grayscale
    # img = cv2.resize(img, (299, 299))  # Resize to 299x299 for InceptionV3
    # img = np.array(img, dtype=np.float32) / 255.0  # Normalize to [0, 1]
    
    # # Since the model expects 3 channels, we stack the grayscale image to create 3 channels
    # img = np.stack([img, img, img], axis=-1)
    
    # img = (img - np.array([0.485, 0.456, 0.406])) / np.array([0.229, 0.224, 0.225])  # Normalize with ImageNet mean and std
    # img = np.transpose(img, (2, 0, 1))  # Transpose to (channels, height, width)
    # img = np.expand_dims(img, axis=0)  # Add batch dimension
    # img = torch.tensor(img, dtype=torch.float32)  # Convert to PyTorch tensor with dtype float32
        # Read the image as grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Resize to 299x299 for InceptionV3
    img = cv2.resize(img, (299, 299))
    
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img = clahe.apply(img)
    
    # Normalize to [0, 1]
    img = np.array(img, dtype=np.float32) / 255.0
    
    # Since the model expects 3 channels, we stack the grayscale image to create 3 channels
    img = np.stack([img, img, img], axis=-1)
    
    # Apply Gamma correction
    gamma = 1.2  # Example gamma value, adjust as necessary
    img = np.power(img, gamma)
    
    # Normalize with ImageNet mean and std
    img = (img - np.array([0.485, 0.456, 0.406])) / np.array([0.229, 0.224, 0.225])
    
    # Transpose to (channels, height, width)
    img = np.transpose(img, (2, 0, 1))
    
    # Add batch dimension
    img = np.expand_dims(img, axis=0)
    
    # Convert to PyTorch tensor with dtype float32
    img = torch.tensor(img, dtype=torch.float32)

    return img

def extract_features(image_path):
    """
    Extract features from a single image using InceptionV3.
    """
    img = preprocess_image(image_path)
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        features = model(img)
    return features.squeeze(0).numpy()

def normalize_features(features):
    """
    Normalize the extracted features.
    """
    return features / np.linalg.norm(features)


/home/jonathan/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [20]:
# # Mobile net v2 (gives to high of scores to everything)
# model = torchvision.models.mobilenet_v2(pretrained=True)

# # Modify the first convolutional layer to accept grayscale images
# model.features[0][0] = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

# def preprocess_image(image_path):
#     """
#     Preprocesses a single grayscale image for MobileNetV2 input.
#     """
#     img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read the image as grayscale
#     img = cv2.resize(img, (224, 224))  # Resize to 224x224 for MobileNetV2
#     img = np.array(img, dtype=np.float32) / 255.0  # Normalize to [0, 1]
    
#     img = np.expand_dims(img, axis=0)  # Add channel dimension
#     img = np.expand_dims(img, axis=0)  # Add batch dimension
#     img = torch.tensor(img, dtype=torch.float32)  # Convert to PyTorch tensor with dtype float32
#     return img

# def extract_features(image_path):
#     """
#     Extract features from a single image using MobileNetV2.
#     """
#     img = preprocess_image(image_path)
#     model.eval()  # Set model to evaluation mode
#     with torch.no_grad():
#         features = model.features(img)
#     return features.squeeze(0).numpy()

# def normalize_features(features):
#     """
#     Normalize the extracted features.
#     """
#     return features / np.linalg.norm(features)

In [21]:
# # DenseNet (gives similar scores to everything)
# model = torchvision.models.densenet121(pretrained=True)

# # Modify the first convolutional layer to accept grayscale images
# model.features.conv0 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# def preprocess_image(image_path):
#     """
#     Preprocesses a single grayscale image for DenseNet input.
#     """
#     img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read the image as grayscale
#     img = cv2.resize(img, (224, 224))  # Resize to 224x224 for DenseNet
#     img = np.array(img, dtype=np.float32) / 255.0  # Normalize to [0, 1]
    
#     img = np.expand_dims(img, axis=0)  # Add channel dimension
#     img = np.expand_dims(img, axis=0)  # Add batch dimension
#     img = torch.tensor(img, dtype=torch.float32)  # Convert to PyTorch tensor with dtype float32
#     return img

# def extract_features(image_path):
#     """
#     Extract features from a single image using DenseNet.
#     """
#     img = preprocess_image(image_path)
#     model.eval()  # Set model to evaluation mode
#     with torch.no_grad():
#         features = model.features(img)
#     return features.squeeze(0).numpy()

# def normalize_features(features):
#     """
#     Normalize the extracted features.
#     """
#     return features / np.linalg.norm(features)


In [22]:
# # Alexnet (gives good scores unconditionally)
# model = torchvision.models.alexnet(pretrained=True)

# # Modify the first convolutional layer to accept grayscale images
# model.features[0] = nn.Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))

# def preprocess_image(image_path):
#     """
#     Preprocesses a single grayscale image for AlexNet input.
#     """
#     img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read the image as grayscale
#     img = cv2.resize(img, (224, 224))  # Resize to 224x224 for AlexNet
#     img = np.array(img, dtype=np.float32) / 255.0  # Normalize to [0, 1]
    
#     img = np.expand_dims(img, axis=0)  # Add channel dimension
#     img = np.expand_dims(img, axis=0)  # Add batch dimension
#     img = torch.tensor(img, dtype=torch.float32)  # Convert to PyTorch tensor with dtype float32
#     return img

# def extract_features(image_path):
#     """
#     Extract features from a single image using AlexNet.
#     """
#     img = preprocess_image(image_path)
#     model.eval()  # Set model to evaluation mode
#     with torch.no_grad():
#         features = model.features(img)
#     return features.squeeze(0).numpy()

# def normalize_features(features):
#     """
#     Normalize the extracted features.
#     """
#     return features / np.linalg.norm(features)

In [92]:
list(model.children())

[BasicConv2d(
   (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
   (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 BasicConv2d(
   (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
   (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 BasicConv2d(
   (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
   (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
 BasicConv2d(
   (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
   (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 BasicConv2d(
   (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
   (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ),
 MaxPo

In [85]:
# best_model_state_dict = model.state_dict()
# torch.save(best_model_state_dict, "best_inception_v3_weights.pth")
best_model_state_dict = torch.load("best2_vgg16_weights.pth")
model.load_state_dict(best_model_state_dict)
model.eval()

Sequential(
  (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [93]:
def compute_similarities_testsets(munich_testset, nk_testset, 
                                  munich_path="munich_test_no_back", 
                                  nk_path="nk_test_no_back"):
    """
    This function takes four arguments: 
    - munich_testset, which contains 5 grayscaled images from the munich database.
    - nk_testset, which contains 5 grayscaled images from the nk collection API.
    - munich path, the path to the directory of the munich images. 
    - nk_path, the path to the directory of the nk images. 
    
    It then computes the feature descriptors for the munich images and all the \
    nk collection images. Afterwards takes the dot-product to get the dot-product similiarity. 
    It then saves the similarity and the two images as key-value pairs in a dictionary. 
    """
    
    similarities = {}
    for nk_img in nk_testset:
        nk_img_path = os.path.join(nk_path, nk_img)
        for munich_img in munich_testset:
            munich_img_path = os.path.join(munich_path, munich_img)
            nk_img_feature_descriptor = normalize_features(extract_features(nk_img_path).flatten())
            munich_img_feature_descriptor = normalize_features(extract_features(munich_img_path).flatten())
            similarity = np.dot(
                nk_img_feature_descriptor,
                munich_img_feature_descriptor
            )
            similarities[(nk_img, munich_img)] = similarity.item()
        
    return similarities

def compute_similarities_img_to_set(nk_img, munich_testset, 
                                    munich_path="mc_no_back", 
                                    nk_path="nk_no_back"):
    similarities = {}
    nk_img_path = os.path.join(nk_path, nk_img)
    
    # Ensure the NK image file exists
    if not os.path.exists(nk_img_path):
        raise FileNotFoundError(f"NK image file {nk_img_path} not found.")
    
    nk_img_feature_descriptor = normalize_features(extract_features(nk_img_path).flatten())
    
    for munich_img in munich_testset:
        munich_img_path = os.path.join(munich_path, munich_img)
        
        # Ensure the Munich image file exists
        if not os.path.exists(munich_img_path):
            raise FileNotFoundError(f"Munich image file {munich_img_path} not found.")
        
        munich_img_feature_descriptor = normalize_features(extract_features(munich_img_path).flatten())
        similarity = np.dot(
            nk_img_feature_descriptor,
            munich_img_feature_descriptor
        )
        similarities[(nk_img, munich_img)] = similarity.item()
        
    return similarities

In [95]:
nk_testset = os.listdir("nk_test_no_back")
munich_testset = os.listdir("munich_test_no_back")

sims_test = compute_similarities_testsets(munich_testset, nk_testset)
sims_test

{('speeltafel_nk.png', 'kast_mccp.jpg'): 0.04646206274628639,
 ('speeltafel_nk.png', 'dressoir_mccp.jpg'): 0.04062419757246971,
 ('speeltafel_nk.png', 'tafel_mccp.jpg'): 0.2207023650407791,
 ('speeltafel_nk.png', 'speeltafel_mccp.png'): 0.4970906674861908,
 ('speeltafel_nk.png', 'stoel_mccp.jpg'): 0.11298052966594696,
 ('dressoir_nk.jpg', 'kast_mccp.jpg'): 0.45805615186691284,
 ('dressoir_nk.jpg', 'dressoir_mccp.jpg'): 0.4778445065021515,
 ('dressoir_nk.jpg', 'tafel_mccp.jpg'): 0.35593485832214355,
 ('dressoir_nk.jpg', 'speeltafel_mccp.png'): 0.08322122693061829,
 ('dressoir_nk.jpg', 'stoel_mccp.jpg'): 0.21342997252941132,
 ('kast_nk.jpg', 'kast_mccp.jpg'): 0.6803843975067139,
 ('kast_nk.jpg', 'dressoir_mccp.jpg'): 0.29671618342399597,
 ('kast_nk.jpg', 'tafel_mccp.jpg'): 0.1898735910654068,
 ('kast_nk.jpg', 'speeltafel_mccp.png'): 0.040228769183158875,
 ('kast_nk.jpg', 'stoel_mccp.jpg'): 0.04464560002088547,
 ('tafel_nk.jpg', 'kast_mccp.jpg'): 0.2868760824203491,
 ('tafel_nk.jpg', 'dre

In [96]:
munich_set = os.listdir("scraped_images_grayscaled_big")
nk_img = ("schilderij_656.jpg")

sims = compute_similarities_img_to_set(nk_img, munich_set, munich_path='scraped_images_grayscaled_big' ,nk_path='nk_collection_schilderij_cleaned')
# print(sims)

In [97]:
def get_table(sims):
    """
    This function takes the output produced by either the compute_similarities \ 
    or compute_similarities_testsets function, and returns a pandas dataframe/table \
    and also saves it in excel.
    """
    
    data = {}
    rows = []

    for key, value in sims.items():
        if key[0] not in data:
            data[key[0]] = []
        if key[1] not in rows:
            rows.append(key[1])
        data[key[0]].append(value)
        
    data = {key[:key.rfind(".")]:value for key, value in data.items()}
    rows = [row[:row.rfind(".")] for row in rows]
        
    df = pd.DataFrame(data, index=rows)
    #df.to_excel('output.xlsx')
    return df.T
    
get_table(sims_test)

,kast_mccp,dressoir_mccp,tafel_mccp,speeltafel_mccp,stoel_mccp
speeltafel_nk,0.046462,0.040624,0.220702,0.497091,0.112981
dressoir_nk,0.458056,0.477845,0.355935,0.083221,0.213430
kast_nk,0.680384,0.296716,0.189874,0.040229,0.044646
tafel_nk,0.286876,0.215521,0.468397,0.200083,0.433952
stoel_nk,0.230801,0.130511,0.491701,0.164806,0.661994


In [98]:
def get_top_10_similarities(similarities):
    """
    Get the top 10 highest similarity values for each NK image from the similarities dictionary.
    
    Parameters:
    - similarities: A dictionary where keys are (NK image name, Munich image name) tuples
                    and values are similarity scores.
                    
    Returns:
    - A dictionary where keys are NK image names and values are lists of tuples
      (Munich image name, similarity) sorted by similarity in descending order.
    """
    top_10_similarities = {}
    
    for nk_img_name in set(key[0] for key in similarities.keys()):
        # Filter similarities for current NK image
        nk_similarities = [(munich_img_name, similarity) for (nk, munich_img_name), similarity in similarities.items() if nk == nk_img_name]
        
        # Sort by similarity in descending order
        sorted_similarities = sorted(nk_similarities, key=lambda x: x[1], reverse=True)
        
        # Get top 10 similarities
        top_10_similarities[nk_img_name] = sorted_similarities[:50]
    
    return top_10_similarities


get_top_10_similarities(sims)


{'schilderij_656.jpg': [('1258_9695_id=cp133005_linz.jpg', 0.8316123485565186),
  ('1142_8573_id=cp133252_linz.jpg', 0.7322022914886475),
  ('1167_8854_id=cp133488_linz.jpg', 0.7194464206695557),
  ('1274_9896_id=cp133853_linz.jpg', 0.6987143158912659),
  ('1208_9251_id=cp130334_linz.jpg', 0.6773865222930908),
  ('1149_8673_id=cp133832_linz.jpg', 0.6762235164642334),
  ('1227_9437_id=cp130649_linz.jpg', 0.6677848696708679),
  ('0460_2354-2_id=cp131030_linz.jpg', 0.6628702282905579),
  ('0644_3869_id=cp131974_linz.jpg', 0.6587140560150146),
  ('0641_3824_id=cp130624_linz.jpg', 0.6557547450065613),
  ('1271_9867_id=cp131364_linz.jpg', 0.6462880969047546),
  ('1324_10770_id=cp131446_linz.jpg', 0.6325349807739258),
  ('1264_9769_id=cp132813_linz.jpg', 0.6267836689949036),
  ('0425_2277-1_id=cp131080_linz.jpg', 0.6239756345748901),
  ('2354_48760_id=cp133580_linz.jpg', 0.6231393218040466),
  ('1271_9868_id=cp177718_badv.jpg', 0.6186420321464539),
  ('1271_9868_id=cp133058_linz.jpg', 0.61790